In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RandomizedSearchCV, ShuffleSplit

properati = pd.read_csv('datos/caba_para_mapa.csv',index_col=0,error_bad_lines=False)

#tenia un nan nose porque
properati.dropna(inplace=True)

<IPython.core.display.Javascript object>

In [13]:
#paso features con strings a numericos
le_barrio = preprocessing.LabelEncoder()
barrios=properati['place_name']
le_barrio.fit(barrios)
properati['place_name'] = le_barrio.transform(barrios)

le_divisa = preprocessing.LabelEncoder()
divisa=properati['currency']
le_divisa.fit(divisa)
properati['currency'] = le_divisa.transform(divisa)

le_tipo = preprocessing.LabelEncoder()
tipos_prop=properati['property_type']
le_tipo.fit(tipos_prop)
properati['property_type'] = le_tipo.transform(tipos_prop)

### Pruebo con np.histogram

In [14]:
cant_buckets = 8

#buckets
cantidad,rango = np.histogram(properati['price_usd_per_m2'], bins=cant_buckets)
properati['categories_by_price']=pd.cut(properati['price_usd_per_m2'],rango,labels=np.arange(cant_buckets))
properati['price_range']=pd.cut(properati['price_usd_per_m2'],rango)
#lo casteo a float porque si no tira error 
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 

#tenia un nan nose porque
properati.dropna(inplace=True)

properati.groupby('categories_by_price').count()

,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,...,expenses,properati_url,description,title,dist_a_subte,dist_a_tren,dist_a_univ,dist_a_villa,dist_a_zona_anegada,price_range
categories_by_price,,,,,,,,,,,,,,,,,,,,,
0.0,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
1.0,155,155,155,155,155,155,155,155,155,155,...,155,155,155,155,155,155,155,155,155,155
2.0,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081,...,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081
3.0,844,844,844,844,844,844,844,844,844,844,...,844,844,844,844,844,844,844,844,844,844
4.0,271,271,271,271,271,271,271,271,271,271,...,271,271,271,271,271,271,271,271,271,271
5.0,61,61,61,61,61,61,61,61,61,61,...,61,61,61,61,61,61,61,61,61,61
6.0,28,28,28,28,28,28,28,28,28,28,...,28,28,28,28,28,28,28,28,28,28
7.0,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10


In [15]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'],properati['dist_a_tren'],properati['dist_a_villa'],\
        properati['dist_a_zona_anegada'],properati['surface_total_in_m2'],properati['place_name'], \
        properati['property_type'])
y = properati['categories_by_price']

### Pruebo con Quantile . NO MEJORO

In [4]:
from sklearn.preprocessing import QuantileTransformer
scaler = QuantileTransformer(copy=False)

In [5]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['price_usd_per_m2']
print X[0]
X=scaler.fit_transform(X,y)
print X[0]

#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

(7.0, 8.5)
[ 0.86886887  0.86636637]
True
True


### Pruebo con RobustScaler. NO MEJORO

In [37]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler(copy=False)

In [38]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['price_usd_per_m2']
print X[0]
X=scaler.fit_transform(X,y)
print X[0]

#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

(7.0, 8.5)
[ 1.1875      0.97560976]
True
True


### Busco hiper-parametros con grid search. Mejor resultado: np.histogram

In [16]:
%%notify
test_tam=0.25
rf = RandomForestClassifier(n_jobs=-1)

bootstrap=[False]
max_features =["auto","sqrt","log2",None]
#max_features =["auto"]

param_grid = {"bootstrap": bootstrap,"max_features":max_features}
custom_cv = ShuffleSplit(n_splits=15, test_size=test_tam)

print ("Test: {0} de {1}".format(round(len(y)*test_tam), len(y)))

grid_search = GridSearchCV(rf, param_grid=param_grid ,cv=custom_cv)
start = time()
grid_search.fit(X, y)

print("GridSearchCV duro %.2f segundos para %d candidatos a hyper-parametros."
    % (time() - start, len(grid_search.cv_results_['params'])))
print("")
score.report_single(grid_search.cv_results_)

Test: 614.0 de 2456
GridSearchCV duro 25.54 segundos para 4 candidatos a hyper-parametros.

Puesto: 1
Promedio training score: 0.990 (std: 0.002)
Promedio validation score: 0.713 (std: 0.016)
Promedio fit time: 0.171s
Hiper-parametros: {'max_features': 'auto', 'bootstrap': False}

Puesto: 2
Promedio training score: 0.990 (std: 0.002)
Promedio validation score: 0.710 (std: 0.018)
Promedio fit time: 0.177s
Hiper-parametros: {'max_features': 'log2', 'bootstrap': False}

Puesto: 3
Promedio training score: 0.990 (std: 0.002)
Promedio validation score: 0.706 (std: 0.017)
Promedio fit time: 0.167s
Hiper-parametros: {'max_features': 'sqrt', 'bootstrap': False}

Puesto: 4
Promedio training score: 0.990 (std: 0.002)
Promedio validation score: 0.676 (std: 0.023)
Promedio fit time: 0.173s
Hiper-parametros: {'max_features': None, 'bootstrap': False}



<IPython.core.display.Javascript object>